In [1]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.9 MB/s eta 0:00:00


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/text_classification/dot/word2vec-t5

/content/drive/MyDrive/text_classification/dot/word2vec-t5


In [5]:
import pandas as pd
from transformers import T5Tokenizer, T5Model
import torch
import jieba

In [10]:
## t5 ##
# Read excel file
data = pd.read_excel('./data.xlsx')
sentences = data['review'].tolist()

def txt_cut(s):
    res = [w for w in jieba.lcut(s) if w.strip()]
    return " ".join(res)

# 对文本进行分词
tokenized_text = []
for i in sentences:
    new = txt_cut(i).split(' ')
    tokenized_text.extend(new)

# Initialize T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5Model.from_pretrained('t5-base')

# Create a list to store all the word vectors
all_vectors = []

# Iterate through each row in the list
for word in tokenized_text:

    # Encode the input
    inputs = tokenizer.encode_plus(word, return_tensors='pt', padding=True, truncation=True)

    # Provide dummy decoder inputs
    inputs['decoder_input_ids'] = torch.tensor([[0]])

    # Pass the input through the model and get the word vectors
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state

    # Append the tensor to all_vectors
    all_vectors.append((word, last_hidden_states))

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
import pandas as pd
from gensim.models import Word2Vec
import jieba
import numpy as np

In [12]:
## word2vec ##

# 初始化Word2Vec模型
model_w = Word2Vec(vector_size=100, window=5, min_count=1)

# 构建词汇表
model_w.build_vocab([tokenized_text])

# 训练模型
model_w.train([tokenized_text], total_examples=model_w.corpus_count, epochs=model_w.epochs)

# 创建一个空的DataFrame来保存词汇和对应的词向量
word_vector_df = pd.DataFrame()

for wd in model_w.wv.index_to_key:
    # 将词向量转化为Series，然后添加到DataFrame中
    word_vector_series = pd.Series([wd] + list(model_w.wv[wd]))
    word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)

<ipython-input-12-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-12-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-12-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-12-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipy

In [15]:
new_word_vectors = []

for word_vector in enumerate(all_vectors):
    word_vector_g = word_vector[1][1].detach().numpy()
    word_vector_g = word_vector_g.reshape(-1, 1)

    ## word2vec
    word_vector_w = model_w.wv[word_vector[1][0]]
    word_vector_w = word_vector_w.reshape(-1, 1)

    ## 新词向量
    new_word_vector = np.multiply(word_vector_g, word_vector_w.T)
    new_word_vectors.append((word, new_word_vector))

In [18]:
# 指定保存文件路径
save_path = 'new_word_vectors.txt'

# 保存单词和新词向量到文件
with open(save_path, 'w') as f:
    for word, word_vector in new_word_vectors:
        word_vector_str = ','.join(map(str, word_vector))
        line = f"{word}: {word_vector_str}\n"
        f.write(line)